# Data Curation
To train a machine learning model from data, that data must first be "curated" to ensure that chemical structures 
and properties are represented consistently.
Curating raw data is a long, detailed process that takes several steps.
**[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** strings need to be standardized, measurements
need to be converted to common units, outliers need to be removed or corrected, 
and replicates need to be combined. These steps are vital to create datasets that
can be used to train useful predictive models. 
Here we will cover some functions in **[AMPL](https://github.com/ATOMScience-org/AMPL)** that will help
you to perform these steps.

- [base_smiles_from_smiles](https://ampl.readthedocs.io/en/latest/utils.html#utils.struct_utils.base_smiles_from_smiles)
- [standardize_relations](https://ampl.readthedocs.io/en/latest/utils.html#utils.data_curation_functions.standardize_relations)
- [compute_negative_log_responses](https://ampl.readthedocs.io/en/latest/utils.html#utils.data_curation_functions.compute_negative_log_responses)
- [remove_outlier_replicates](https://ampl.readthedocs.io/en/latest/utils.html#utils.curate_data.remove_outlier_replicates)
- [aggregate_assay_data](https://ampl.readthedocs.io/en/latest/utils.html#utils.curate_data.aggregate_assay_data)

These are just a few of the steps needed to curate a dataset.

## Import Standard Data Science Packages
To use **[AMPL](https://github.com/ATOMScience-org/AMPL)**, or to do almost anything else with data, you'll need to become familiar with the popular packages 
**[pandas](https://pandas.pydata.org/)**, 
**[numpy](https://numpy.org/)**, 
**[matplotlib](https://matplotlib.org/)** and 
**[seaborn](https://seaborn.pydata.org/)**. 
When you installed **[AMPL](https://github.com/ATOMScience-org/AMPL)** you will have installed these packages as well, so you simply need to import them here.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem

In [ ]:
# MMP Data
from rdkit.Chem import PandasTools

train_df = pd.read_csv("/Users/echun/repos/DILI-rotation/datasets_ST/MMP/sr-mmp.smiles", sep=r"\s+", header=None, names=["base_rdkit_smiles", "compound_id", "active"])
test_df=PandasTools.LoadSDF('/Users/echun/repos/DILI-rotation/datasets_ST/MMP/tox21_10k_challenge_test.sdf', idName='ID', molColName='ROMol', includeFingerprints=False, isomericSmiles=True, smilesName='base_rdkit_smiles', embedProps=False, removeHs=True, strictParsing=True, sanitize=True)
test_df

In [3]:
filter_test = test_df[['Compound Batch ID', 'base_rdkit_smiles', 'SR-MMP']]
test_cleaned = filter_test.dropna()
test_cleaned = test_cleaned.rename(columns={'Compound Batch ID': 'compound_id', 'SR-MMP': 'active'})
test_cleaned

,compound_id,base_rdkit_smiles,active
0,NCGC00261443-01,CNc1ncnc2c1ncn2[C@@H]1O[C@H](CO)C(O)[C@H]1O,0
3,NCGC00261266-01,Br.Cc1onc(O)c1CC(N)C(=O)O,0
5,NCGC00261559-01,c1ccc2c(c1)CC(N1CCN(c3cccc4c3OCCO4)CC1)C2,0
6,NCGC00261121-01,Cl.Oc1cc2c(cc1O)C(c1ccccc1)CNCC2,0
7,NCGC00261374-01,Cl.N=C(N)c1ccc(CS(=O)(=O)F)cc1,0
...,...,...,...
289,NCGC00261317-01,COc1cc(CC(=O)O)ccc1O,0
291,NCGC00261292-01,Cl.NCCc1c[nH]c2ccc(O)cc12,0
293,NCGC00260828-01,CC1=C(/C=C/C(C)=C/C=C/C(C)=C\C=C\C=C(C)/C=C/C=...,0
294,NCGC00260687-01,COc1ccc2[nH]c3c(c2c1)CCNC3,0


In [4]:
score_df = pd.read_csv("/Users/echun/repos/DILI-rotation/datasets_ST/MMP/tox21_10k_challenge_score.smiles", sep=r"\s+")
score_df_active = pd.read_csv('/Users/echun/repos/DILI-rotation/datasets_ST/MMP/tox21_10k_challenge_score.txt', sep=r'\s+')
filter_score_active = score_df_active[['Sample','SR-MMP']]
filter_score_active = filter_score_active[~filter_score_active.isin(['x']).any(axis=1)]
merged_score = pd.merge(score_df, filter_score_active, on='Sample', how='outer')
merged_score = merged_score[['#SMILES','Sample','SR-MMP']]
merged_score = merged_score.rename(columns={'Sample': 'compound_id', 'SR-MMP': 'active', '#SMILES': 'base_rdkit_smiles'})

concatenated_df = pd.concat([train_df, test_cleaned, merged_score], ignore_index=True)
concatenated_df.to_csv("new_mmp.csv", index=False)

Clean this csv first with adhoc notebook

## Read the Data
We've prepared an example dataset containing **[$K_i$](https://en.wikipedia.org/wiki/Ligand_\(biochemistry\)#Receptor/ligand_binding_affinity)** values for inhibitors of 
the **[SLC6A3](https://www.ebi.ac.uk/chembl/target_report_card/CHEMBL238/)** 
dopamine transporter collected from **[ChEMBL](https://www.ebi.ac.uk/chembl/)**. This dataset is 
simpler than most that we find in the wild, but it will let us concisely 
demonstrate some **[AMPL](https://github.com/ATOMScience-org/AMPL)** curation tools. The first step of data curation is to read the raw data into a Pandas data frame.

In [2]:
# Read in data
meas="MMP"
dir="up"
raw_df = pd.read_csv(f'/Users/echun/repos/DILI-rotation/datasets_ST/MMP/MMP_hits_up_class_cur.csv')

In [3]:
# Check the number of rows and columns in the dataset
raw_df.shape

(6179, 5)

In [4]:
# List the column names
raw_df.columns.values

array(['compound_id', 'base_rdkit_smiles', 'relation', 'active',
       'new_smiles'], dtype=object)

In [ ]:
#merge if needed
ogdf = pd.read_csv("/Users/echun/repos/DILI-rotation/datasets/training_data/ROS_hits_up_class_cur.csv")
ogdf = ogdf[['compound_id','base_rdkit_smiles','active']]
merged = pd.concat([ogdf, concatenated_df], ignore_index=True)
merged['relation']='='
merged['active'] = pd.to_numeric(merged['active'], errors='coerce')
merged = merged[merged["compound_id"] != "NCGC00261231-01"]

merged


In [31]:
from atomsci.ddm.utils.curate_data import remove_outlier_replicates, aggregate_assay_data
clean['relation']='='

clean = aggregate_assay_data(clean, 
                             value_col='active',
                             output_value_col='active',
                             id_col='compound_id',
                             smiles_col='base_rdkit_smiles',
                             relation_col='relation',
                             label_actives=False,
                             verbose=True
                        )

clean

0 entries in input table are missing SMILES strings


[10:15:24] Can't kekulize mol.  Unkekulized atoms: 3 10
[10:15:26] WARNING: not removing hydrogen atom without neighbors
[10:15:26] WARNING: not removing hydrogen atom without neighbors


7390 unique SMILES strings are reduced to 6902 unique base SMILES strings


,compound_id,base_rdkit_smiles,relation,active
0,NCGC00257743-01,BrC(Br)Br,,0.0
1,NCGC00258125-01,BrC(Br)C(Br)(Br)Br,,0.0
2,NCGC00257918-01,BrC(Br)C(Br)Br,,0.0
3,NCGC00260276-01,BrC/C=C/CBr,,0.0
4,NCGC00257050-01,BrC1CCC(Br)C(Br)CCC(Br)C(Br)CCC1Br,,1.0
...,...,...,...,...
6897,NCGC00256860-01,c1coc(Cn2cccc2)c1,,0.0
6898,NCGC00160467-01,c1csc(C2(N3CCCCC3)CCCCC2)c1,,0.0
6899,NCGC00256662-01,c1csc(SSc2cccs2)c1,,0.0
6900,NCGC00257608-01,c1nc[n-]n1,,0.0


In [32]:
clean[clean.active.between(0,1, inclusive='neither')].shape #101
clean = clean[(clean["active"] == 0) | (clean["active"] == 1)].copy()

clean

,compound_id,base_rdkit_smiles,relation,active
0,NCGC00257743-01,BrC(Br)Br,,0.0
1,NCGC00258125-01,BrC(Br)C(Br)(Br)Br,,0.0
2,NCGC00257918-01,BrC(Br)C(Br)Br,,0.0
3,NCGC00260276-01,BrC/C=C/CBr,,0.0
4,NCGC00257050-01,BrC1CCC(Br)C(Br)CCC(Br)C(Br)CCC1Br,,1.0
...,...,...,...,...
6897,NCGC00256860-01,c1coc(Cn2cccc2)c1,,0.0
6898,NCGC00160467-01,c1csc(C2(N3CCCCC3)CCCCC2)c1,,0.0
6899,NCGC00256662-01,c1csc(SSc2cccs2)c1,,0.0
6900,NCGC00257608-01,c1nc[n-]n1,,0.0


## Standardize SMILES
The **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** grammar allows the same chemical structure to be represented by many different **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** strings. In addition, measurements may be performed on compounds with different salt groups or with radioisotope labels, which we treat as equivalent to the base compounds. **[AMPL](https://github.com/ATOMScience-org/AMPL)** provides a **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** standardization function, `base_smiles_from_smiles`, that removes salt groups and isotopes and returns a unique **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** string for each base compound structure. This step simplifies the machine learning problem by ensuring each compound is represented with the same set of features and multiple measurements on the same compound can be grouped together. 

> **Note:** *The input to `base_smiles_from_smiles` must be a list; numpy arrays and pandas Series objects must be converted with the tolist function.*

In [ ]:
from atomsci.ddm.utils.struct_utils import base_smiles_from_smiles
# Since the base_smiles_from_smiles function can be slow, we specify the workers=8 argument to divide the work across 8 threads.
concatenated_df['base_rdkit_smiles'] = base_smiles_from_smiles(concatenated_df.base_rdkit_smiles.tolist(), workers=8)

In [15]:
raw_df.loc[raw_df.compound_id=='SPID634', 'base_rdkit_smiles']=base_smiles_from_smiles(['C[C@H](CCCC(C)(C)O)[C@H]1CC[C@H]2\C(CCC[C@@]12C)=C\C=C1\C[C@@H](O)C[C@H](O)C1=C'])
raw_df['relation']='='

In [16]:
raw_df.base_rdkit_smiles.nunique()

802

For this dataset there are 1830 unique **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** that are standardized to 1823 unique base **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)**. It is common for two different **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** strings to be standardized to the same value. From now on we will use `base_rdkit_smiles` to represent compound structures.

## Remove Outliers and Aggregate Replicate Measurements
The final step is to remove outliers and aggregate (average) replicate measurements on the same compounds. The function `remove_outlier_replicates` is a simple filter that groups measurements by compound, computes the median of each group, and removes values that differ more than `max_diff_from_median` units from the median. When the measurements are very spread out relative to `max_diff_from_median`, *all* the rows for a compound may be deleted from the dataset. The default setting ($1.0$) generally works well for $pK_i$ values.

The function `aggregate_assay_data` replaces multiple replicate measurements for each compound with a single aggregate value. Usually this is simply the average over the replicates, but if the dataset contains both censored and uncensored values for a compound, the function computes a maximum likelihood estimate that takes the censoring into account.


In [17]:
raw_df['active'].value_counts().sort_index()


active
0.0    548
1.0    251
Name: count, dtype: int64

In [18]:
from atomsci.ddm.utils.curate_data import remove_outlier_replicates, aggregate_assay_data

curated_df = remove_outlier_replicates(raw_df, id_col='compound_id',
                                response_col='active',
                                max_diff_from_median=1.0)

curated_df = aggregate_assay_data(raw_df, 
                             value_col='active',
                             output_value_col='active',
                             id_col='compound_id',
                             smiles_col='base_rdkit_smiles',
                             relation_col='relation',
                             label_actives=False,
                             verbose=True
                        )
print("Original data shape: ", raw_df.shape)
print("Curated data shape: ", curated_df.shape)

curated_df = curated_df.drop('relation', axis=1)
curated_df.head()

/Users/echun/repos/AMPL/atomsci/ddm/utils/curate_data.py:535: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filt_df = gby.apply(filter_outliers)


0 entries in input table are missing SMILES strings
802 unique SMILES strings are reduced to 802 unique base SMILES strings
Original data shape:  (802, 4)
Curated data shape:  (802, 4)


,compound_id,base_rdkit_smiles,active
0,SPID236,C#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21C,0.0
1,SPID231,C#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21CC,1.0
2,SPID339,C#CCN(C)[C@H](C)Cc1ccccc1,1.0
3,FOUWCSDKDDHKQP,C#CCN1C(=O)COc2cc(F)c(N3C(=O)C4=C(CCCC4)C3=O)cc21,1.0
4,SPID475,C#CCN[C@@H]1CCc2ccccc21,0.0


In [20]:
raw_df[raw_df.base_rdkit_smiles.isin(curated_df[curated_df.active.between(0.0, 1.0, inclusive='neither')].base_rdkit_smiles)].sort_values('base_rdkit_smiles')

,compound_id,base_rdkit_smiles,active,relation


In [21]:
# check dupes
if meas=='CellCount':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,1]))
elif meas=='GSH':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [0,1,1,1,1,1]))
elif meas=='MitoStruct':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,0,1,1]))
elif meas=='MMP':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,0,1,1,0,0,1,1]))
elif meas=='NucArea':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,0,1,1,0]))
elif meas=='NucMask':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,1]))
elif meas=='ROS':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,1,1,1]))
     
curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'active']=curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'].map(map_dict)


In [54]:
print("Duplicate SMILES:", raw_df["base_rdkit_smiles"].duplicated().sum())
print("Duplicate compound_id:", raw_df["compound_id"].duplicated().sum())


Duplicate SMILES: 1388
Duplicate compound_id: 0


Finally, we save the curated dataset to a CSV file. 

In [24]:
curated_df.to_csv(f'/Users/echun/repos/DILI-rotation/AMPL_models/datasets/training_data/ROS_hits_up_class_cur.csv', index=False)

In [ ]:
messy = pd.read_csv(f'../datasets/training_data/MMP_tox21_cur_with_moe_descriptors.csv')
clean = pd.read_csv(f'../datasets/training_data/MMP_tox21_cur.csv')

unmatched = clean.merge(
    messy,
    how="left",
    on=["compound_id", "base_rdkit_smiles", "active"],
    indicator=True
).query('_merge == "left_only"')

unmatched

# moe_df = clean.merge(messy, how="left", on=["compound_id", "base_rdkit_smiles", 'active'])
# moe_df.shape
# moe_df.to_csv(f'../datasets/training_data/MMP_tox21_curMoe.csv', index=False)

# Create MT dataset

In [26]:
meas="ROS"
dir="up"
df=pd.read_csv(f'/Users/echun/repos/DILI-rotation/AMPL_models/datasets/training_data/ROS_hits_up_class_cur.csv')
# df=df.drop(columns=['relation'])
suffix1 = '_ROS'

for file in os.listdir('/Users/echun/repos/DILI-rotation/AMPL_models/datasets/training_data'):
    if 'cur' in file:
        if 'ROS' in file:
            pass
        else:
            df2=pd.read_csv(f'/Users/echun/repos/DILI-rotation/AMPL_models/datasets/training_data/{file}')
            try:
                df2=df2.drop(columns=['relation'])
            except:
                pass
            suffix2 = '_'+file.split('_')[0]
            print(file, suffix2)
            df=df.merge(df2, on=['compound_id','base_rdkit_smiles'], how='outer', suffixes=(suffix1, suffix2))
            suffix1=suffix2
df=df.rename(columns={'active':f'active{suffix2}'})
df

MMP_hits_up_class_cur.csv _MMP
CellCount_hits_dn_class_cur.csv _CellCount
MitoStruct_hits_up_class_cur.csv _MitoStruct
GSH_hits_up_class_cur.csv _GSH
NucMask_hits_up_class_cur.csv _NucMask
NucArea_hits_dn_class_cur.csv _NucArea


,compound_id,base_rdkit_smiles,active_ROS,active_MMP,active_CellCount,active_MitoStruct,active_GSH,active_NucMask,active_NucArea
0,AAEVYOVXGOFMJO,CSc1nc(NC(C)C)nc(NC(C)C)n1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AEMFNILZOJDQLW,C[C@]12CCC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@]2(C)C(...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AIMMSOZBPYFASU,COc1cc(OC)nc(NC(=O)[N-]S(=O)(=O)c2ncccc2OCC(F)...,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AKTXOQVMWSFEBQ,CC(C)(C)c1cc(/C=C2\SC(=N)NC2=O)cc(C(C)(C)C)c1O,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ALWUKGXLBSQSMA,CCCCCCC1(C)CCC(=O)O1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7251,ZNOLGFHPUIJIMJ,COP(=S)(OC)Oc1ccc([N+](=O)[O-])c(C)c1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7252,ZOTRFGNOTDLOAU,CCC(CC)Nc1ccc(C)c(C)c1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7253,ZUBPKHVCBGWWGO,Cc1cc(SC2=C(O)C[C@@](CCc3ccc(N)cc3)(C(C)C)OC2=...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7254,ZXVONLUNISGICL,Cc1cc([N+](=O)[O-])cc([N+](=O)[O-])c1O,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df[df.base_rdkit_smiles.duplicated(keep=False)].sort_values('base_rdkit_smiles').to_csv(f'/Users/echun/repos/DILI-rotation/AMPL_models/datasets/dupes.csv', index=False)

In [29]:
df=df[~df.base_rdkit_smiles.duplicated(keep=False)]
dupes=pd.read_csv(f'/Users/echun/repos/DILI-rotation/AMPL_models/datasets/dupes_cur.csv')

df=pd.concat([df,dupes], ignore_index=True)

In [32]:
df.to_csv(f'/Users/echun/repos/DILI-rotation/AMPL_models/datasets/cell_health_MT_supps.csv', index=False)

In [33]:
print("Duplicate SMILES:", df["base_rdkit_smiles"].duplicated().sum())
print("Duplicate compound_id:", df["compound_id"].duplicated().sum())


Duplicate SMILES: 0
Duplicate compound_id: 0
